In [ ]:
import pandas as pd
import sqlite3

import plotly.express as px

MOVIE_DATABASE_PATH = "../data/movies.db"
conn = sqlite3.connect(MOVIE_DATABASE_PATH)

In [7]:
movies     = pd.read_sql("SELECT * FROM movies", conn)
directors  = pd.read_sql("SELECT * FROM directors", conn)
genres     = pd.read_sql("SELECT * FROM genres", conn)
movie_dirs = pd.read_sql("SELECT * FROM movie_directors", conn)
movie_gens = pd.read_sql("SELECT * FROM movie_genres", conn)

# Attach names
md = movie_dirs.merge(directors, left_on="director_id", right_on="id")
mg = movie_gens.merge(genres, left_on="genre_id", right_on="id")

# Aggregate lists
dir_lists = md.groupby("page_title")["name"].apply(list).rename("directors")
gen_lists = mg.groupby("page_title")["name"].apply(list).rename("genres")

# Join back to movies
df = (movies
      .merge(dir_lists, how="left", left_on="page_title", right_index=True)
      .merge(gen_lists, how="left", left_on="page_title", right_index=True))

In [8]:
null_tokens = {"None", "none", "NULL", "null", ""}
df = df.replace(list(null_tokens), pd.NA)

for col in ["budget", "box_office", "rotten_tomatoes", "metacritic", "running_time"]:
    df[col] = pd.to_numeric(df[col], errors="coerce").astype("Int64")  # nullable ints

In [9]:
df["release_year"] = (
    pd.to_datetime(df["release_date"], errors="coerce")
      .dt.year
      .astype("Int64")
)

In [10]:
original_percentage_top = []

for year in range(1900, 2025):
    top = (
        df[df["release_year"] == year]
        .sort_values("budget", ascending=False)
        [["movie_title", "budget", "classification"]]
        .head(10)
    )
    if top.empty:
        original_percentage_top.append(0)
        continue

    mask = top["classification"].isin(["original", "adapted"])
    pct = mask.sum() / len(top)
    original_percentage_top.append(pct)

fig = px.line(original_percentage_top, 
    x=range(1900, 2025), 
    y=original_percentage_top,
    title="Percentage of Top 10 Grossing Movies that are Original"
)
fig.update_layout(
    xaxis_title="Year",
    yaxis_title="Original share",
    yaxis_range=[0, 1],
)
fig.show()